In [1]:
import os
import sys
import time
import calendar
import datetime

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import scipy as sp
from scipy import stats
# import tensorflow as tf

In [2]:
from scipy.special import rel_entr
from scipy.stats import wasserstein_distance

In [3]:

real_df = pd.read_csv(f"real_data/final_df--uk.csv", parse_dates=["datetime"])
real_df["days_passed"] = real_df["td"]

real_df["Amount"] = real_df.amount
real_df.amount = real_df.amount.apply(abs)


gbr = real_df[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()

for fname in ['generated_data/gen_v2b__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-nd__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-nc__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-v__nld_4-dm_128-nh_2-i_2-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/tg_0-hd_24-nl_4-bs_64--len_20.csv',
'generated_data/dg_data_new.csv']:
    
    # print(fname)
    
    gdf = pd.read_csv(fname, parse_dates=['datetime'])
    
    
    
    gdf["month"] = gdf["datetime"].dt.month
    gdf["day"]   = gdf["datetime"].dt.day
    gdf["dow"]  =  gdf["datetime"].dt.dayofweek
    gdf["year"]  = gdf["datetime"].dt.year


# gbr = real_df[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()
    gbg = gdf[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()

    wd = wasserstein_distance(gbr.raw_amount, gbg.raw_amount)
    print(wd)
    # print("Wasserstein between real is", wd, "\n\n")
    
    

541.8145429885301
814.8736765705576
1418.7622942078024
1402.2994419699533
1460.7071648290867
1051.152169134766


In [4]:
541.8
814.8
1418
1402
1460
1051

877.3

# Joint date, tcode

In [6]:
date_feature = "dow"


def set_tcode_date(df, date_feature):
    df[f'tcode_{date_feature}'] = df.tcode + "---" + df[date_feature].astype(str)
    

def get_counts(df, field):
    return df[date_feature].value_counts().reset_index().rename(columns={'index':'value', date_feature:'counts'})


field = f'tcode_{date_feature}'


set_tcode_date(real_df, date_feature)




# real_ngrams = create_ngramcount_df(real_df, n, field)
# real_ngrams = real_df[field].value_counts().reset_index().rename(columns={'index':'ngram'})

real_ngrams = get_counts(real_df, date_feature)

print("With field =", date_feature, "\n\n")


for fname in ['generated_data/gen_v2b__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-nd__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-nc__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-v__nld_4-dm_128-nh_2-i_2-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/tg_0-hd_24-nl_4-bs_64--len_20.csv',
'generated_data/dg_data_new.csv']:
    
    # print(fname)
    
    df = pd.read_csv(fname, parse_dates=['datetime'])
    
    
    
    df["month"] = df["datetime"].dt.month
    df["day"]   = df["datetime"].dt.day
    df["dow"]  =  df["datetime"].dt.dayofweek
    df["year"]  = df["datetime"].dt.year

    
    
  
    set_tcode_date(df, date_feature)
    

    
    # create combo_df, which contains counts of all ngrams for both datasets (note: it omits any ngrams which do not occur in either dataset)
    gen_ngrams = get_counts(df, date_feature)
    combo_df = pd.merge(real_ngrams, gen_ngrams, on="value", how="outer", suffixes=("_real", "_gen")).fillna(0.0)


    N_obs_real = real_ngrams["counts"].sum()
    N_obs_gen = gen_ngrams["counts"].sum()

    
    # set ps
    combo_df["ps_real"] = combo_df["counts_real"] / N_obs_real 
    combo_df["ps_gen"] = combo_df["counts_gen"] / N_obs_gen 
    

        
        
    # compute jsd (note: contribution to jsd from any ngram not in either dataset is 0)
    combo_df["ps_mid"] = (combo_df["ps_real"] + combo_df["ps_gen"])/2
    kl_real_M = sum(rel_entr(combo_df["ps_real"], combo_df["ps_mid"])) 
    kl_gen_M = sum(rel_entr(combo_df["ps_gen"], combo_df["ps_mid"]))

    jsd = (kl_real_M + kl_gen_M)/2
    
    print(f"{jsd:0.3f}")
        


With field = dow 


0.008
0.062
0.024
0.063
0.077
0.061


In [7]:
date_feature = "day"


def set_tcode_date(df, date_feature):
    df[f'tcode_{date_feature}'] = df.tcode + "---" + df[date_feature].astype(str)
    

def get_counts(df, field):
    return df[date_feature].value_counts().reset_index().rename(columns={'index':'value', date_feature:'counts'})


field = f'tcode_{date_feature}'


set_tcode_date(real_df, date_feature)




# real_ngrams = create_ngramcount_df(real_df, n, field)
# real_ngrams = real_df[field].value_counts().reset_index().rename(columns={'index':'ngram'})

real_ngrams = get_counts(real_df, date_feature)

print("With field =", date_feature, "\n\n")


for fname in ['generated_data/gen_v2b__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-nd__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-nc__nld_4-dm_128-nh_2-i_1-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/gen_v2b-v__nld_4-dm_128-nh_2-i_2-dr_0__1-opt_adam-lwi_0-bs_64--vf1-len_20-v2.csv',
 'generated_data/tg_0-hd_24-nl_4-bs_64--len_20.csv',
'generated_data/dg_data_new.csv']:
    
    # print(fname)
    
    df = pd.read_csv(fname, parse_dates=['datetime'])
    
    
    
    df["month"] = df["datetime"].dt.month
    df["day"]   = df["datetime"].dt.day
    df["dow"]  =  df["datetime"].dt.dayofweek
    df["year"]  = df["datetime"].dt.year

    
    
  
    set_tcode_date(df, date_feature)
    

    
    # create combo_df, which contains counts of all ngrams for both datasets (note: it omits any ngrams which do not occur in either dataset)
    gen_ngrams = get_counts(df, date_feature)
    combo_df = pd.merge(real_ngrams, gen_ngrams, on="value", how="outer", suffixes=("_real", "_gen")).fillna(0.0)


    N_obs_real = real_ngrams["counts"].sum()
    N_obs_gen = gen_ngrams["counts"].sum()

    
    # set ps
    combo_df["ps_real"] = combo_df["counts_real"] / N_obs_real 
    combo_df["ps_gen"] = combo_df["counts_gen"] / N_obs_gen 
    

        
        
    # compute jsd (note: contribution to jsd from any ngram not in either dataset is 0)
    combo_df["ps_mid"] = (combo_df["ps_real"] + combo_df["ps_gen"])/2
    kl_real_M = sum(rel_entr(combo_df["ps_real"], combo_df["ps_mid"])) 
    kl_gen_M = sum(rel_entr(combo_df["ps_gen"], combo_df["ps_mid"]))

    jsd = (kl_real_M + kl_gen_M)/2
    
    print(f"{jsd:0.3f}")
        


With field = day 


0.034
0.044
0.061
0.042
0.089
0.039


# New - can go in own nb - compare datasets

In [9]:
real_df_cz = pd.read_csv("../../code/banksformer_v2a/stored_data/final_df--czech.csv", parse_dates=['datetime'])

In [18]:
# real_df_cz = pd.read_csv('/Users/kylenickerson/Desktop/verafin_proj_2/czech-banking-data/lpetrocelli-some-translatedreformatted-czech-banking-data/data/data/new_transaction.csv')

In [16]:
# real_df_cz = pd.read_csv("../../code/banksformer_v2a/data/tr_by_acct_w_age.csv")

In [10]:
real_df_uk = pd.read_csv(f"real_data/final_df--uk.csv", parse_dates=["datetime"])

In [11]:
real_df_cz.datetime.min(),real_df_cz.datetime.max()

(Timestamp('1993-01-01 00:00:00'), Timestamp('1998-12-31 00:00:00'))

In [12]:
real_df_cz.datetime.max() - real_df_cz.datetime.min() 

Timedelta('2190 days 00:00:00')

In [13]:
real_df_uk.datetime.max() - real_df_uk.datetime.min() 

Timedelta('54 days 14:00:00')

In [19]:
len(real_df_cz), len(real_df_uk)

(1056320, 100000)

In [21]:
len(real_df_cz)/10**6, len(real_df_uk)/10**5

(1.05632, 1.0)

In [39]:
real_df_uk.datetime.min(),real_df_uk.datetime.max()

(Timestamp('2017-04-01 06:00:00'), Timestamp('2017-05-25 20:00:00'))

In [23]:
def set_raw_amount(df):
    df["raw_amount"] = df.amount * df.type.apply(lambda x: -1 if x.upper() == "DEBIT" else 1)  
    

In [24]:
set_raw_amount(real_df_uk)
real_df_uk.describe()

,unnamed: 0,index,amount,balance,age,month,day,dow,year,dtme,td,raw_amount
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.0,100000.000000,100000.000000,100000.000000,100000.0,100000.000000,100000.000000,100000.000000
mean,50000.500000,12.074430,72.764891,1288.336661,-1.0,4.550130,15.892100,2.707520,2017.0,14.658030,1.921480,286.621223
std,28867.657797,9.129026,765.004653,1589.131034,0.0,0.497483,8.112066,1.776375,0.0,8.318097,2.957411,713.003755
min,1.000000,0.000000,-2171.000000,-5602.380000,-1.0,4.000000,1.000000,0.000000,2017.0,1.000000,0.000000,-9.140000
25%,25000.750000,5.000000,-43.942500,192.765000,-1.0,4.000000,10.000000,1.000000,2017.0,8.000000,0.000000,14.420000
50%,50000.500000,10.000000,-25.000000,1192.250000,-1.0,5.000000,17.000000,3.000000,2017.0,13.000000,1.000000,32.000000
75%,75000.250000,19.000000,-9.710000,2232.865000,-1.0,5.000000,22.000000,4.000000,2017.0,21.000000,3.000000,54.382500
max,100000.000000,49.000000,6745.500000,10213.070000,-1.0,5.000000,29.000000,5.000000,2017.0,30.000000,27.000000,6745.500000


In [21]:
real_df_uk.describe()

,unnamed: 0,index,amount,balance,age,month,day,dow,year,dtme,td,raw_amount
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.0,100000.000000,100000.000000,100000.000000,100000.0,100000.000000,100000.000000,100000.000000
mean,50000.500000,12.074430,72.764891,1288.336661,-1.0,4.550130,15.892100,2.707520,2017.0,14.658030,1.921480,72.764891
std,28867.657797,9.129026,765.004653,1589.131034,0.0,0.497483,8.112066,1.776375,0.0,8.318097,2.957411,765.004653
min,1.000000,0.000000,-2171.000000,-5602.380000,-1.0,4.000000,1.000000,0.000000,2017.0,1.000000,0.000000,-2171.000000
25%,25000.750000,5.000000,-43.942500,192.765000,-1.0,4.000000,10.000000,1.000000,2017.0,8.000000,0.000000,-43.942500
50%,50000.500000,10.000000,-25.000000,1192.250000,-1.0,5.000000,17.000000,3.000000,2017.0,13.000000,1.000000,-25.000000
75%,75000.250000,19.000000,-9.710000,2232.865000,-1.0,5.000000,22.000000,4.000000,2017.0,21.000000,3.000000,-9.710000
max,100000.000000,49.000000,6745.500000,10213.070000,-1.0,5.000000,29.000000,5.000000,2017.0,30.000000,27.000000,6745.500000


In [28]:
[real_df_cz.raw_amount.min(),real_df_cz.raw_amount.max()]

[-87400.0, 74812.0]

In [35]:
real_df_cz.raw_amount.describe().reset_index().iloc[1:]

,index,raw_amount
1,mean,186.639739
2,std,11213.527089
3,min,-87400.000000
4,25%,-3019.000000
5,50%,-14.600000
6,75%,200.000000
7,max,74812.000000


In [37]:
real_df_cz.tcode.describe().reset_index().iloc[1:]

,index,tcode
1,unique,16
2,top,DEBIT__CASH WITHDRAWAL__nan
3,freq,274059


In [29]:
[real_df_uk.raw_amount.min(),real_df_uk.raw_amount.max()]

[-9.14, 6745.5]

In [36]:
real_df_uk.raw_amount.describe().reset_index().iloc[1:]

,index,raw_amount
1,mean,286.621223
2,std,713.003755
3,min,-9.140000
4,25%,14.420000
5,50%,32.000000
6,75%,54.382500
7,max,6745.500000


In [38]:
real_df_uk.tcode.describe().reset_index().iloc[1:]

,index,tcode
1,unique,44
2,top,Groceries - Food & Drinks__Card Transaction__D...
3,freq,24779


In [31]:
real_df_cz.groupby("account_id").size().describe()

count    4500.000000
mean      234.737778
std       126.849443
min         9.000000
25%       133.000000
50%       208.000000
75%       330.000000
max       675.000000
dtype: float64

In [32]:
real_df_uk.groupby("account_id").size().describe()

count    5000.000000
mean       20.000000
std        10.148783
min         2.000000
25%        11.000000
50%        17.500000
75%        29.000000
max        50.000000
dtype: float64